In [ ]:
import re
import os
import requests
import numpy as np
from time import sleep
from itertools import product

# Template Notebook

In [ ]:
BASE_URL = "https://online-go.com/api/v1/games"

ALPHA = 'abcdefghijklmnopqrstuvwxyz'
B = "⚫"
W = "⚪"
PT = "╶╴"
SP = "╺╸"

BLACK = '\033[30m'      # ]
GRAY = '\033[90m'       # ]
YELLOW = '\033[33m'     # ]
BG_YELLOW = '\033[43m'  # ]
RESET = '\033[0m'       # ]

In [ ]:
def retrieve_sgf(game_url: str) -> str:
    """Retrieve the SGF text from the given game url or id

    Parameters
    ----------
    game_url
        the url or game id of an OGS game

    Returns
    -------
    the Smart Game Format text data for the given game
    """

    game_id = game_url.strip("https://online-go.com/game/")
    request_url = os.path.join(BASE_URL, game_id, 'sgf')

    r = requests.get(request_url)
    return r.content.decode('utf-8')


def read_sgf(sgf: str) -> tuple:
    """Parse a Smart Game Format file into metadata and game moves

    Parameters
    ----------
    sgf
        string data of the SGF file

    Returns
    -------
    a tuple containing metadata and game moves
    """

    text = sgf.strip(')').split(';')
    meta = dict(re.findall(r'(\w+)\[(.*?)\]\n?', text[1]))
    moves = [tuple(m.strip('\n()]').split('[')) for m in text[2:]]

    return meta, moves

In [ ]:
class Board:
    """Representation of a Go board

    Parameters
    ----------
    size : int
        The size of the board dictates the length and width
        Typically 9, 13, or 19
    debug : bool
        a flag for enabling debugging behavior

    Attributes
    ----------
    state : np.array
        State of the current board represented in integers
        0 = empty space, 1 = black stone, 2 = white stone
    """

    def __init__(self, size: int, debug: bool = False) -> None:
        self.size = size
        self.debug = debug
        self.state = np.zeros((size,size), dtype=object)

    def get_stones(self, player: int|None = None) -> np.ndarray:
        """Get stone objects from board state

        Parameters
        ----------
        player
            which player's stones to return, if none return all stones

        Returns
        -------
        List of stone objects
        """
        if player is not None:
            return self.state[self.state == player]
        return self.state[self.state > 0]

    @property
    def stones(self) -> np.ndarray:
        """Return all stones on the board

        Returns
        -------
        List of Stone objects
        """
        return self.get_stones()

    def process_groups(self) -> None:
        """Find groups of stones and remove those with no liberties"""

        def recursive_find_group(stone, group=None) -> set:
            """Recursively find groups of stones

            Parameters
            ----------
            stone
                a stone to be added to the group
            group
                the current group of stones
            Returns
            -------
            a unique set of the stones in a group with the starting stone
            """

            group = set() if group is None else group
            if stone in group:
                return set()
            group.add(stone)
            for con in stone.connections:
                recursive_find_group(con, group)
            return group

    def play(self, player: int, x: int, y: int) -> None:
        """Play a stone on the board

        Parameters
        ----------
        player
            1 for black, 2 for white
        x
            The column on which to play
        y
            The row on which to play
        """

        self.state[y, x] = Stone(self, player, np.array((y, x)))
        self.process_groups()

        groups = []
        stones = set(self.get_stones())
        while stones:
            group = recursive_find_group(stones.pop())
            groups.append(group)
            stones = stones - group

            liberties = len(set.union(*[s.liberties for s in group]))
            if liberties == 0:
                remove = np.array([s.location for s in group])
                self.state[remove[:, 0], remove[:, 1]] = 0

    def plaintext_board(self) -> str:
        """Create a terminal-printable plain text board, including colors

        Returns
        -------
        Plain text representation of the current board state
        """

        star_points = np.zeros((self.size,self.size), dtype=int)
        corners = [j for i in range(3) if (j:=((s:=2+(self.size>9))+(2*s*i))) < self.size]
        pts = [(f:=self.size//2, f)] + list(product(corners, repeat=2))
        star_points[*zip(*pts)] = -1

        board = self.state.copy()
        mask = ~self.state.astype(bool)
        board[mask] = star_points[mask]

        if self.debug:
            joined = ' '.join([str(i) for i in range(self.size)]).upper()
        else:
            joined = ' '.join(list(ALPHA.replace('i', '')[:self.size])).upper()
        rows = [col_label:=f"{YELLOW}{(d:='-' * s)}{BLACK}{joined} {YELLOW}{d}"]

        for r, input_row in enumerate(board.astype(int), 1):
            row = ''.join([(PT,B,W,SP)[i] for i in input_row])

            if self.debug:
                num = str(r - 1)
            else:
                num = str(self.size - r + 1)

            lnum = num.rjust(int(len(str(self.size))))
            rnum = num.ljust(int(len(str(self.size))))
            rows.append(f'{BLACK}{lnum} {GRAY}{row} {BLACK}{rnum}')

        rows.append(col_label)
        rows = [f'{BG_YELLOW}{row}{RESET}' for row in rows]

        return '\n'.join(rows)

    def __str__(self) -> str:
        return self.plaintext_board()

    def __repr__(self) -> str:
        return self.plaintext_board()

In [ ]:
class Stone:
    """Representation of a Go stone

    Parameters
    ----------
    board : Board
        the Board object on which the stone is played
    color : int
        the color/player of the stone
        1 for black, 2 for white
    location : np.array
        x, y location of the stone on the board
    """

    def __init__(self, board: Board, color: int, location: np.ndarray) -> None:
        self.board = board
        self.color = color  # 1 is black, 2 is white
        self.location = location

    @property
    def neighbors(self) -> tuple:
        """Determines the locations and values corresponding to each space neighboring this stone

        Returns
        -------
        tuple containing an array of locations and an array of values of neighbors
        """
        neighbor_locs = np.array(((0, 1), (1, 0), (0, -1), (-1, 0))) + self.location
        oob = (neighbor_locs < 0) | (neighbor_locs >= self.board.size)
        neighbor_locs = neighbor_locs[~oob.any(axis=1)]
        neighbor_vals = self.board.state[neighbor_locs[:, 0], neighbor_locs[:, 1]]

        return neighbor_locs, neighbor_vals

    @property
    def connections(self) -> list:
        """Returns the locations of each friendly connection on the board"""

        locs, vals = self.neighbors
        conns = locs[vals == self.color]
        return list(self.board.state[conns[:, 0], conns[:, 1]])

    @property
    def liberties(self) -> set:
        """Returns the locations of each open liberty on the board"""

        locs, vals = self.neighbors
        return set(map(tuple, locs[vals < 1]))

    def __int__(self) -> int:
        return self.color

    def __index__(self) -> int:
        return self.__int__()

    def __hash__(self) -> int:
        y, x = self.location
        return hash(f'{x}{y}')

    def __gt__(self, other) -> bool:
        return self.color > other

    def __lt__(self, other) -> bool:
        return self.color < other

    def __eq__(self, other) -> bool:
        return self.color == other

    def __str__(self) -> str:
        return f'{(B, W)[self.color-1]} {self.location}'

    def __repr__(self) -> str:
        return f'Stone(Board, {self.color}, {self.location})'

In [ ]:
b = Board(9, debug=True)
plays = [
    (2, 3, 4), (2, 4, 3), (2, 4, 4), (2, 4, 5),
    (1, 5, 3), (1, 5, 4), (1, 5, 5), (1, 4, 2),
    (1, 4, 6), (1, 3, 3), (1, 3, 5)
]

for player, x, y in plays:
    b.play(player, x, y)
    # time.sleep(0.5)

print(b)

In [ ]:
def recursive_find_group(stone, group=None) -> set:
    group = set() if group is None else group
    if stone in group:
        return set()
    group.add(stone)
    for con in stone.connections:
        recursive_find_group(con, group)
    return group

In [ ]:
# starting_stone = b.stones[4]  # white stone 4, 3
starting_stone = b.stones[6]
result = recursive_find_group(starting_stone)
result
b

In [ ]:
b.stones[0]
locs, vals = b.stones[0].neighbors
for loc, val in zip(locs, vals):
    print(loc, val)

In [ ]:
groups = []
stones = set(b.get_stones())
while stones:
    group = recursive_find_group(stones.pop())
    groups.append(group)
    stones = stones - group

len(groups)

In [ ]:
b.play(1, 2, 4)

group = groups[2]
liberties = len(set.union(*[s.liberties for s in group]))

print('Total liberties:', liberties)

In [ ]:
if liberties == 0:
    remove = np.array([s.location for s in group])
    b.state[remove[:, 0], remove[:, 1]] = 0

print(b)

In [ ]:
groups = []
stones = set(b.get_stones())
while stones:
    group = recursive_find_group(stones.pop())
    groups.append(group)
    stones = stones - group

    liberties = len(set.union(*[s.liberties for s in group]))
    if liberties == 0:
        remove = np.array([s.location for s in group])
        b.state[remove[:, 0], remove[:, 1]] = 0

In [ ]:
b = Board(9, debug=True)
plays = [
    (1, 3, 4), (2, 4, 4),
    (1, 4, 3), (2, 5, 3),
    (1, 4, 5), (2, 5, 5),
    (1, 5, 4)
]

In [ ]:
for player, x, y in plays:
    b.play(player, x, y)
    sleep(1)
    print(b)

In [ ]:
game = "https://online-go.com/game/68039230"
sgf = retrieve_sgf(game)
meta, moves = read_sgf(sgf)

size = int(meta['SZ'])

In [ ]:
b = Board(size)

for player, move in moves:
    player = 'BW'.index(player) + 1

    if move:  # an empty move is a pass
        x, y = (ALPHA.index(c) for c in move)

        b.play(player, x, y)
        print(b)
        sleep(0.15)